# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [3]:
key = '006179570C'
number = driver.find_element_by_name('mcrdata')
number.send_keys(key)
time.sleep(1)
number.send_keys(Keys.RETURN)

In [4]:
company_info = {}
company_info_raw = driver.find_elements_by_tag_name('table')
# There are multiple tables, I only care about the 4th and 5th. 
# I combine the two to make it easier to loop through them 
segments = company_info_raw[3].find_elements_by_tag_name('td') + company_info_raw[4].find_elements_by_tag_name('td')
# the info I want is at the following positions:
# (Except for some companies, but I'll solve this with the function later)
stuff_i_want = [2,4,6,9]
for n in stuff_i_want:
    # I can get the key name from the data by splitting the string at the ':', except for the address
    company_info[segments[n].text.split(':')[0].strip()] = segments[n].text.split(':')[1].strip()
company_info['address'] = segments[11].text.replace('\n','').split('al:')[1]
company_info



{'Name': 'B.D. SMITH TOWING',
 'Owner/Officer': 'BRANDT SMITH / OWNER',
 'Phone': '8173330706',
 'Status': 'Expired',
 'address': '13619 BRETT JACKSON RD.FORT WORTH, TX. 76179'}

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [5]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')
key = '006179570C'
number = driver.find_element_by_name('mcrdata')
number.send_keys(key)
time.sleep(1)
number.send_keys(Keys.RETURN)
company_info = {}
company_info_raw = driver.find_elements_by_tag_name('table')
stuff_i_want = [2,4,6,9]
segments = company_info_raw[3].find_elements_by_tag_name('td') + company_info_raw[4].find_elements_by_tag_name('td')
for n in stuff_i_want:
    company_info[segments[n].text.split(':')[0].strip()] = segments[n].text.split(':')[1].strip()
company_info['address'] = segments[11].text.replace('\n','').split('al:')[1]
company_info


{'Name': 'B.D. SMITH TOWING',
 'Owner/Officer': 'BRANDT SMITH / OWNER',
 'Phone': '8173330706',
 'Status': 'Expired',
 'address': '13619 BRETT JACKSON RD.FORT WORTH, TX. 76179'}

# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [6]:
df = pd.read_csv('trucks-subset.csv')

In [7]:
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [8]:
def get_url(num):
    return 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=' + str(num)

In [9]:
pd.set_option('display.max_colwidth', -1)
df['TDLR Number'].apply(get_url)

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
Name: TDLR Number, dtype: object

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [10]:
df['url'] = df['TDLR Number'].apply(get_url)

In [11]:
df.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [12]:
def get_cominfo(url):
    driver.get(url)
    company_info = {}
    company_info_raw = driver.find_elements_by_tag_name('table')
    stuff_i_want = [2,4,6,9]
    segments = company_info_raw[3].find_elements_by_tag_name('td') + company_info_raw[4].find_elements_by_tag_name('td')
    # Things get a bit weird for some of the companies in the larger set, as they have 2 owners.
    # I copy the info of Owner #2 to the dictionary and delete the list item (and the following empty item, too)
    # Now the company loops like all the others
    if 'Owner' in segments[6].text:
        company_info['Owner/Officer_2'] = segments[6].text.split(':')[1].strip()
        del segments[6]
        del segments[6]
    # Let's also make it a bit more failsafe, because why not
    try:    
        for n in stuff_i_want:
            company_info[segments[n].text.split(':')[0].strip()] = segments[n].text.split(':')[1].strip()
        company_info['address'] = segments[11].text.replace('\n','').split('al:')[1]
    except:
        print('Theres a problem with this page: {}'.format(url))
    return pd.Series(company_info)
    

In [13]:
df['url'].apply(get_cominfo)

,Name,Owner/Officer,Phone,Status,address
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN STBONHAM, TX. 75418"
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.FORT WORTH, TX. 76179"
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RDCANYON, TX. 79015"


## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [14]:
df = df['url'].apply(get_cominfo).join(df)
df.head()

,Name,Owner/Officer,Phone,Status,address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN STBONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RDCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [15]:
df.to_csv('tow-trucks-extended.csv', index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [16]:
df2 = pd.read_csv('tow-trucks-extended.csv')
df2.head()

,Name,Owner/Officer,Phone,Status,address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN STBONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RDCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [17]:
df_all = pd.read_csv('tow-trucks.csv')

In [18]:
df_all['url'] = df_all['TDLR Number'].apply(get_url)
df_all.shape

(20, 2)

In [20]:
df_all = df_all['url'].apply(get_cominfo).join(df_all)
df_all.shape

(20, 8)

In [21]:
df_all.to_csv('tow-trucks-extended.csv')
df_all2 = pd.read_csv('tow-trucks-extended.csv')
df_all2.head(3)

,Unnamed: 0,Name,Owner/Officer,Phone,Status,address,Owner/Officer_2,TDLR Number,url
0,0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN STBONHAM, TX. 75418",NaN,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.FORT WORTH, TX. 76179",NaN,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RDCANYON, TX. 79015",NaN,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
